
# Experiments Model Free Analysis

This is a testing notebook: building out the first set of functionality for the *_summary_plots_and_figures.py_* class.

Statistical inquiry into the aggregate behaviour of **all** Tasks.

A more detailed analysis will be conducted to investigate the relationships between variables/experiments & *wcst*, however the data structures employed here will be leveraged for this subsequent analysis.


---------
```
Zach Wolpe
zachcolinwolpe@gmail.com
04 June 2022
```
---------




In [2]:
# from encode_processed_data import encode_data
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.express as px
import scipy.stats as stats
import pandas as pd
import numpy as np
import pickle
import sys
import os
import re

sys.path.append('../process data/')

In [3]:
# with open('../data objects/batch_processing_object.pkl', 'rb') as file2:
#     bp = pickle.load(file2)

# ---- fetch data object ----x
with open('../data objects/batch_processing_object_with_encodings.pkl', 'rb') as file2:
    ed = pickle.load(file2)

   
ed.__dict__.keys()

/Users/zachwolpe/miniforge3/envs/hipercog/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



dict_keys(['raw', 'summary_table', 'fitts_summary_stats', 'corsi_summary_stats', 'navon_summary_stats', 'nback_summary_stats', 'demographics_plot', 'demographics'])

In [4]:
ed.describe_data()



        ------------------------------------------------------------------
            self.path            : raw data loc
            self.metadata        : mturk metadata
            self.mapping         : reference table
            self.data_times      : reference times table
            self.participants    : list of participant identifiers
            self.parti_code      : list of participant codes
            self.n               : total number of samples
            self.wcst_paths      : paths to wcst  raw data
            self.nback_paths     : paths to nback raw data
            self.corsi_paths     : paths to corsi raw data
            self.fitts_paths     : paths to fitts raw data
            self.navon_paths     : paths to navon raw data
            self.wcst_data       : wcst  dataframe
            self.nback_data      : nback dataframe
            self.corsi_data      : corsi dataframe
            self.fitts_data      : fitts dataframe
            self.navon_data    

In [5]:
ed.clean_data_info()



                WCST - Wisconsin Card Sorting Task                                                  DataFrame: ed.raw.wcst_date
            ---------------------------------------------------------------------------------------------------------------------------
            
                participant                     : key               : participant ID
                card_no                         : categorical       : the card shown
                correct_card                    : categorical       : the card that should be clicked of the top four on screen      
                correct_persevering             : categorical       : the card that would be clicked if the participant is persevering
                seq_no                          : numeric           : trial number
                rule                            : categorical       : matching rule  
                card_shape                      : categorical       : current card shape
                card_num

In [6]:
# ----- all categories descriptors -----x
cats_demographics   = ['gender_a', 'handedness_a', 'education_a', 'age_group']
cats_navon          = [('level_of_target', '')]

# ---- add numerical descriptors ----x
num_demographics = ['age_a','income_a', 'computer_hours_a', 'mean_reation_time_ms']
num_nback = [('block_number', ''), ('score', 'mean'), ('score', 'std'), ('status', 'mean'), ('status', 'std'), ('miss', 'mean'), 
            ('miss', 'std'), ('false_alarm', 'mean'), ('false_alarm', 'std'), ('reaction_time_ms', 'mean'), ('reaction_time_ms', 'std')]
num_navon = [('correct', 'mean'), ('correct',  'std'), ('too_slow', 'mean'), ('too_slow',  'std'), 
            ('reaction_time_ms', 'mean'), ('reaction_time_ms',  'std')]
num_corsi = [('highest_span',  'max'), ('status', 'mean'), ('status',  'std')]
num_fitts = [('delta', 'mean'), ('delta',  'std'), ('status', 'mean')]


# ---- user selected menus ---x

ed.summary_table.head()

nback_status  nback_reaction_time_ms  fitts_mean_deviation  \
participant                                                               
100934.0            0.900               2225.9125                 -5.70   
100934.0            0.900               2225.9125                 -5.70   
100934.0            0.900               2225.9125                 -5.70   
103322.0            0.925               2373.6250               -203.05   
103322.0            0.925               2373.6250               -203.05   

             corsi_block_span navon_level_of_target  navon_perc_correct  \
participant                                                               
100934.0                  4.0                global            0.166667   
100934.0                  4.0                 local            0.461538   
100934.0                  4.0                  none            0.480000   
103322.0                  6.0                global            0.923077   
103322.0                  6.0                 local            0.916667   

             navon_reaction_time_ms  wcst_accuracy  wcst_RT  \
participant                                                   
100934.0                 664.583333           0.84  1795.16   
100934.0                 597.769231           0.84  1795.16   
100934.0                 622.960000           0.84  1795.16   
103322.0                 959.769231           0.87  1406.27   
103322.0                 999.750000           0.87  1406.27   

             demographics_age_a demographics_gender_a  \
participant                                             
100934.0                   28.0                  male   
100934.0                   28.0                  male   
100934.0                   28.0                  male   
103322.0                   51.0                  male   
103322.0                   51.0                  male   

            demographics_handedness_a demographics_education_a  \
participant                                                      
100934.0                        right               university   
100934.0                        right               university   
100934.0                        right               university   
103322.0                        right               university   
103322.0                        right               university   

             demographics_income_a  demographics_computer_hours_a  \
participant                                                         
100934.0                       7.0                           25.0   
100934.0                       7.0                           25.0   
100934.0                       7.0                           25.0   
103322.0                       6.0                           82.0   
103322.0                       6.0                           82.0   

            demographics_age_group  demographics_mean_reation_time_ms  
participant                                                            
100934.0                     25-34                       11453.571429  
100934.0                     25-34                       11453.571429  
100934.0                     25-34                       11453.571429  
103322.0                     45-54                        5294.428571  
103322.0                     45-54                        5294.428571

# Fitts Law

In [7]:
def fitts_law_deviation(n=10, data=ed.fitts_summary_stats[[('delta','mean')]]):

    # ---- compute bins ----x
    def compute_fitts_bins(data, n=n):
        rng = np.linspace(min(data[('delta','mean')]), max(data[('delta','mean')]), n)
        data['bin'] = 'na'; data['group'] = 'na'
        for r in range(n-1): 
            data.loc[(data[('delta','mean')] > rng[r]) & (data[('delta','mean')] <= rng[r+1]), 'bin']   = str(round(rng[r+1])) + '-' + str(round(rng[r])) 
            data.loc[(data[('delta','mean')] > rng[r]) & (data[('delta','mean')] <= rng[r+1]), 'group'] = r+1
        return(data)
    data = compute_fitts_bins(data=data)
    
    # ---- plot ----x
    trace  = go.Histogram(x=data[('delta','mean')], marker_color='#4ca3dd')
    layout = go.Layout( title='Mean Fitts Law Deviation', xaxis={'title':'deviation from expected (Fitts Law Predicted) RT'}, yaxis={'title':'Frequency'}, 
                        template='none', width=700, height=500)
    fig    = go.Figure(data=trace, layout=layout)

    return({'data':data, 'figure':fig})


fitts = fitts_law_deviation()
fitts['figure']

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#4ca3dd'},
              'type': 'histogram',
              'x': array([   -5.7 ,  -203.05,  -342.65, ...,  -316.5 , -1168.  , -1144.45])}],
    'layout': {'height': 500,
               'template': '...',
               'title': {'text': 'Mean Fitts Law Deviation'},
               'width': 700,
               'xaxis': {'title': {'text': 'deviation from expected (Fitts Law Predicted) RT'}},
               'yaxis': {'title': {'text': 'Frequency'}}}
})

In [8]:
data = ed.nback_summary_stats
data.head()
data.describe()

participant trial_counter       score                  status  \
                             count        mean         std        mean   
count     254.000000         254.0  254.000000  254.000000  254.000000   
mean   546111.299213          80.0    0.309941    0.461811    0.749459   
std    262095.942611           0.0    0.052761    0.022570    0.181790   
min    100934.000000          80.0    0.162500    0.371236    0.275000   
25%    318693.750000          80.0    0.275000    0.449331    0.637500   
50%    528567.500000          80.0    0.312500    0.466437    0.762500   
75%    790053.000000          80.0    0.337500    0.475840    0.912500   
max    998593.000000          80.0    0.475000    0.502525    1.000000   

                         miss             false_alarm              \
              std        mean         std        mean         std   
count  254.000000  254.000000  254.000000  254.000000  254.000000   
mean     0.377182    0.201378    0.353936    0.108563    0.256624   
std      0.120850    0.116152    0.155167    0.112175    0.139752   
min      0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.284349    0.125000    0.332805    0.025000    0.157110   
50%      0.428236    0.225000    0.420217    0.062500    0.243589   
75%      0.475840    0.287500    0.455452    0.171875    0.379582   
max      0.503155    0.462500    0.501737    0.437500    0.499208   

      reaction_time_ms               
                  mean          std  
count       254.000000   254.000000  
mean       2200.262943   899.856611  
std         599.220358   376.484246  
min         229.950000     0.000000  
25%        2017.896875   834.831805  
50%        2256.950000  1065.933795  
75%        2525.978125  1159.145169  
max        3000.000000  1273.568641

In [9]:
def scatter_plot(data, xvar, yvar, group_var=False, xlab='', ylab='', title='', cols=px.colors.qualitative.Pastel):
    

    if not group_var: 
        traces = [go.Scatter(x=data[xvar], y=data[yvar], mode='markers', marker_color=cols[0])]
        layout = go.Layout( title=title, xaxis={'title':xlab}, yaxis={'title':ylab}, template='none')
    else:
        traces = []; c=0
        for g in np.unique(data[group_var]):
            c += 1
            dt = data.loc[data[group_var]==g,]
            traces.append(go.Scatter(x=dt[xvar], y=dt[yvar], mode='markers', marker_color=cols[c], name=round(g)))
        layout = go.Layout( title=title, xaxis={'title':xlab}, yaxis={'title':ylab}, template='none', legend_title_text='Trend')
    fig = go.Figure(data=traces, layout=layout)
    return fig

scatter_plot(data=data, group_var='block_number', xvar=('reaction_time_ms', 'mean'), yvar=('status', 'mean'), xlab='Reaction Time (ms)', ylab='% Correct', title='Performance by Reaction Time (RT)')


    

KeyError: 'block_number'

In [10]:
# ------------------------------- function: summmary dataset -------------------------------x

def return_summary_data(ed=ed):
    # ----- NBack -----x
    x = ed.nback_summary_stats.groupby('participant').agg({
        ('status', 'mean'): ['mean'],
        ('reaction_time_ms', 'mean'): ['mean']
    })
    x.columns = ['nback_status', 'nback_reaction_time_ms']
    nback = x 

    # ----- Fitts -----x
    x = ed.fitts_summary_stats[[('participant', ''), ('delta','mean')]].set_index('participant')
    x.columns = ['fitts_mean_deviation']
    fitts = x

    # ----- corsi -----x
    x = ed.corsi_summary_stats[[('participant', ''), ('highest_span','max')]].set_index('participant')
    x.columns = ['corsi_block_span']
    corsi = x

    # ----- Navon -----x
    x = ed.navon_summary_stats[[('participant', ''), ('level_of_target',''), ('correct','mean'), ('reaction_time_ms', 'mean')]].set_index('participant')
    x.columns = ['navon_level_of_target', 'navon_perc_correct', 'navon_reaction_time_ms']
    navon = x

    # ----- wcst ----x
    def compute_performance_trial_bins(n_bins=10, wcst_data=ed.raw.wcst_data):
        """Return: DataFrame capturing the performance per n_bins triasl"""

        # ---- add trial number ----x
        xx = []; df = wcst_data
        [xx.append((i%100)+1) for i in range(df.shape[0])]
        df['trial_no'] = xx 

        # ---- status==1 --> correct
        t = np.linspace(0,100,num=n_bins+1).tolist(); c=0

        for tt in t[1:]:
            c +=1
            x = df.loc[df['trial_no'] < tt,].groupby(['participant', 'status']).agg({
            'participant':              ['count'],
            'reaction_time_ms':         ['mean', 'std'],
            'perseverance_error':       ['mean'],
            'not_perseverance_error':   ['mean']
            }).reset_index()
            x['percentages'] = x[('participant', 'count')]/tt
            x['trials']      = str(round(t[c-1])) + '-' + str(round(t[c]))
            x['trials_2']    = t[c]
            if c==1:    data=x
            else:       data=data.append(other=x)

        # if x>0 --> perseverance_error > not_perseverance_error --> main error=perseverance_error
        data['main_error'] = np.where(data['perseverance_error'] - data['not_perseverance_error'] > 0, 'perserverance errors', 'non perserverance errors')

        return(data)

    x = compute_performance_trial_bins(n_bins=1)
    x = x.loc[x['status']==1, [('participant', ''), ('reaction_time_ms', 'mean'), ('percentages', '')]].set_index('participant')
    x.columns = ['wcst_RT', 'wcst_accuracy']
    wcst = x

    # ---- demograpics ----x
    x = ed.demographics.set_index('participant')
    x.columns = ['demographics_' + xx for xx in x.columns]
    demo = x


    # ---- Join ----x
    df = nback
    for d in [nback, fitts, corsi, navon, wcst, demo][1:]:
        df = df.join(d, how='outer')

    # ------ discrete vars ------x
    categorical_vars = ['navon_level_of_target', 'demographics_gender_a','demographics_handedness_a', 'demographics_education_a','demographics_age_group']

    # ------ continuous vars ------x
    continuous_vars  = ['nback_status', 'nback_reaction_time_ms', 'fitts_mean_deviation', 'corsi_block_span', 'navon_perc_correct', 
                        'navon_reaction_time_ms', 'wcst_RT', 'wcst_accuracy', 'demographics_age_a', 'demographics_income_a', 
                        'demographics_computer_hours_a', 'demographics_mean_reation_time_ms']

    # ----- fix datatypes -----x
    df[categorical_vars] = df[categorical_vars].astype('string')
    df[continuous_vars]  = df[continuous_vars].astype('float')

    return df


df = return_summary_data()
df.head()


nback_status  nback_reaction_time_ms  fitts_mean_deviation  \
participant                                                               
100934.0            0.900               2225.9125                 -5.70   
100934.0            0.900               2225.9125                 -5.70   
100934.0            0.900               2225.9125                 -5.70   
103322.0            0.925               2373.6250               -203.05   
103322.0            0.925               2373.6250               -203.05   

             corsi_block_span navon_level_of_target  navon_perc_correct  \
participant                                                               
100934.0                  4.0                global            0.166667   
100934.0                  4.0                 local            0.461538   
100934.0                  4.0                  none            0.480000   
103322.0                  6.0                global            0.923077   
103322.0                  6.0                 local            0.916667   

             navon_reaction_time_ms      wcst_RT  wcst_accuracy  \
participant                                                       
100934.0                 664.583333  1661.590361           0.83   
100934.0                 597.769231  1661.590361           0.83   
100934.0                 622.960000  1661.590361           0.83   
103322.0                 959.769231  1363.674419           0.86   
103322.0                 999.750000  1363.674419           0.86   

             demographics_age_a demographics_gender_a  \
participant                                             
100934.0                   28.0                  male   
100934.0                   28.0                  male   
100934.0                   28.0                  male   
103322.0                   51.0                  male   
103322.0                   51.0                  male   

            demographics_handedness_a demographics_education_a  \
participant                                                      
100934.0                        right               university   
100934.0                        right               university   
100934.0                        right               university   
103322.0                        right               university   
103322.0                        right               university   

             demographics_income_a  demographics_computer_hours_a  \
participant                                                         
100934.0                       7.0                           25.0   
100934.0                       7.0                           25.0   
100934.0                       7.0                           25.0   
103322.0                       6.0                           82.0   
103322.0                       6.0                           82.0   

            demographics_age_group  demographics_mean_reation_time_ms  
participant                                                            
100934.0                     25-34                       11453.571429  
100934.0                     25-34                       11453.571429  
100934.0                     25-34                       11453.571429  
103322.0                     45-54                        5294.428571  
103322.0                     45-54                        5294.428571

# Plots + Stats 

# ScatterPlot

In [11]:
def scatter_plot(data, xvar, yvar, group_var=False, xlab='', ylab='', title='', cols=px.colors.qualitative.Pastel):

    if not group_var: 
        data = data[[xvar, yvar]].dropna()
        traces = [go.Scatter(x=data[xvar], y=data[yvar], mode='markers', marker_color=cols[0])]
        layout = go.Layout( title=title, xaxis={'title':xlab}, yaxis={'title':ylab}, template='none')
    else:
        data = data[[xvar, yvar, group_var]].dropna()
        traces = []; c=0
        for g in data[group_var].unique():
            c += 1
            dt = data.loc[data[group_var]==g,]
            traces.append(go.Scatter(x=dt[xvar], y=dt[yvar], mode='markers', marker_color=cols[c], name=g))
        layout = go.Layout( title=title, xaxis={'title':xlab}, yaxis={'title':ylab}, template='none', legend_title_text='Trend')
    fig = go.Figure(data=traces, layout=layout)
    return fig

scatter_plot(data=df, group_var='demographics_gender_a', xvar='nback_status', yvar='corsi_block_span', xlab='NBack performance', ylab='Corsi Block Span', title='NBack vs Corsi Performance')

    

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': 'rgb(246, 207, 113)'},
              'mode': 'markers',
              'name': 'male',
              'type': 'scatter',
              'x': array([0.9   , 0.9   , 0.9   , ..., 0.9625, 0.9625, 0.9625]),
              'y': array([4., 4., 4., ..., 7., 7., 7.])},
             {'marker': {'color': 'rgb(248, 156, 116)'},
              'mode': 'markers',
              'name': 'female',
              'type': 'scatter',
              'x': array([0.925 , 0.925 , 0.925 , ..., 0.5625, 0.5625, 0.5625]),
              'y': array([6., 6., 6., ..., 0., 0., 0.])},
             {'marker': {'color': 'rgb(220, 176, 242)'},
              'mode': 'markers',
              'name': 'other',
              'type': 'scatter',
              'x': array([0.8375, 0.8375, 0.8375, 1.    , 1.    , 1.    ]),
              'y': array([6., 6., 6., 7., 7., 7.])}],
    'layout': {'legend': {'title': {'text': 'Trend'}},
               'template': '...',
               'title': {'text': 'NBack vs Corsi Performance'},
               'xaxis': {'title': {'text': 'NBack performance'}},
               'yaxis': {'title': {'text': 'Corsi Block Span'}}}
})

In [12]:
scatter_plot(data=df, group_var='demographics_education_a', xvar='wcst_RT', yvar='wcst_accuracy', title='WCST Performance vs WCST RT', cols=px.colors.qualitative.Safe)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': 'rgb(204, 102, 119)'},
              'mode': 'markers',
              'name': 'university',
              'type': 'scatter',
              'x': array([1661.59036145, 1661.59036145, 1661.59036145, ..., 1697.93333333,
                          1697.93333333, 1189.859375  ]),
              'y': array([0.83, 0.83, 0.83, ..., 0.75, 0.75, 0.64])},
             {'marker': {'color': 'rgb(221, 204, 119)'},
              'mode': 'markers',
              'name': 'high school',
              'type': 'scatter',
              'x': array([1407.98666667, 1407.98666667, 1407.98666667, 2208.28767123,
                          2208.28767123, 2208.28767123, 1253.82051282, 1253.82051282,
                          1253.82051282, 1137.12048193, 2056.76744186, 2056.76744186,
                          2056.76744186, 1313.51388889, 1313.51388889, 1313.51388889,
                          1699.82608696, 1699.82608696, 1699.82608696, 1426.62195122,
                          1914.09090909, 1914.09090909, 1914.09090909, 1471.21428571,
                          1471.21428571, 1471.21428571, 1150.6       , 1150.6       ,
                          1150.6       , 1445.64772727, 1445.64772727, 1445.64772727,
                          1306.97333333, 1638.32142857, 1638.32142857, 1638.32142857,
                          1656.19753086, 1656.19753086, 1656.19753086, 1335.34782609,
                          1335.34782609, 1335.34782609, 1846.25      , 1846.25      ,
                          1846.25      , 1503.2       , 1503.2       , 1503.2       ,
                          1639.71428571, 1639.71428571, 1639.71428571, 1214.        ,
                          1214.        , 1214.        , 1387.12359551, 1387.12359551,
                          1387.12359551, 1243.39285714, 1243.39285714, 1243.39285714,
                          1445.81609195, 1445.81609195, 1445.81609195, 1254.85714286,
                          1254.85714286, 1254.85714286, 1099.3253012 , 1106.54878049,
                          1106.54878049, 1106.54878049, 1354.34615385, 1354.34615385,
                          1354.34615385, 1796.02631579, 1796.02631579, 1796.02631579,
                          1092.625     , 1092.625     , 1092.625     , 1410.79104478,
                          1410.79104478, 1410.79104478, 1125.28571429, 1125.28571429,
                          1125.28571429, 1117.52380952, 1117.52380952, 1117.52380952,
                          1541.9382716 , 1541.9382716 , 1541.9382716 ,  877.72941176,
                           877.72941176,  877.72941176, 1520.59090909, 1520.59090909,
                          1520.59090909, 1368.20987654, 1368.20987654, 1368.20987654,
                          1173.46590909, 1173.46590909, 1173.46590909, 1800.92      ,
                          1800.92      , 1800.92      , 1827.16666667, 1827.16666667,
                          1827.16666667, 1649.22666667, 1649.22666667, 1649.22666667,
                          1421.93589744, 1421.93589744, 1421.93589744, 2148.78846154,
                          2148.78846154, 2148.78846154, 1183.39726027, 1183.39726027,
                          1183.39726027,  921.58024691,  921.58024691,  921.58024691,
                          2739.17142857, 2739.17142857, 2739.17142857, 1323.31395349,
                          1323.31395349, 1323.31395349, 1095.98717949, 1095.98717949,
                          1095.98717949, 1292.03947368, 1292.03947368, 1292.03947368,
                          1215.87209302, 1215.87209302, 1215.87209302, 1265.02325581,
                          1265.02325581, 1265.02325581]),
              'y': array([0.75, 0.75, 0.75, 0.73, 0.73, 0.73, 0.78, 0.78, 0.78, 0.83, 0.86, 0.86,
                          0.86, 0.72, 0.72, 0.72, 0.69, 0.69, 0.69, 0.82, 0.88, 0.88, 0.88, 0.84,
                          0.84, 0.84, 0.6 , 0.6 , 0.6 , 0.88, 0.88, 0.88, 0.75, 0.84, 0.84, 0.84,
                          0.81, 0.81, 0.81, 0.23, 0.23, 0.23, 0.76, 0.76, 0.76, 0.75, 0.75,

In [14]:
# px.colors.qualitative.swatches()

In [15]:
from plotly.colors import n_colors
from plotly.subplots import make_subplots

import plotly.express as px



def distribution_plot(data, xvar, nbinsx=10, opacity=1, group_var=False, xlab='', ylab='', title='', cols=['#A56CC1', '#A6ACEC', '#63F5EF', 'steelblue', 'darkblue']):
    """Distribution of Variable 1"""
    if title=='': 
        if group_var: title = 'Distribution of ' + str(xvar) + ' by ' + str(group_var)
        else: title = 'Distribution of ' + str(xvar)

    if not group_var: 
        data   = data[[xvar]].dropna()
        traces = [go.Histogram(x=data[xvar], marker_color=cols[0], nbinsx=nbinsx, opacity=opacity)]
        layout = go.Layout(title=title, xaxis={'title':xlab}, yaxis={'title':ylab}, template='none')
        fig    = go.Figure(data=traces, layout=layout)
        return fig
    else:
        fig = make_subplots(rows=2, cols=1, subplot_titles=('', ''))
        data   = data[[xvar, group_var]].dropna()
        traces = []; c=0; RTs = []
        for g in data[group_var].unique():
            dt = data.loc[data[group_var]==g,]
            RTs.append(dt[xvar])


        
            c += 1
            fig.add_trace(go.Histogram(x=dt[xvar], nbinsx=nbinsx, marker_color=cols[c], name=g, opacity=opacity), row=2, col=1)

        # ---- sort lists ----x
        # RTs.append(dt[xvar])
        srt = np.argsort([np.mean(r) for r in RTs])
        RT  = [RTs[s] for s in srt]

        # ---- create figure: violin plots ----x
        c=-1
        for nm, rt in zip(data[group_var].unique(), RT):
            c+=1
            fig.add_trace(go.Violin(
                showlegend=False, y=rt, name=nm, box_visible=True,
                meanline_visible=True, fillcolor=cols[c], line_color=cols[-1]), row=1, col=1)
        
        
        fig.update_layout(title_text=title, height=700, template='none')
        return fig

    

    

    


distribution_plot(data=df, nbinsx=50, xvar='wcst_RT', group_var='demographics_education_a')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#A6ACEC'},
              'name': 'university',
              'nbinsx': 50,
              'opacity': 1,
              'type': 'histogram',
              'x': array([1661.59036145, 1661.59036145, 1661.59036145, ..., 1697.93333333,
                          1697.93333333, 1189.859375  ]),
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'marker': {'color': '#63F5EF'},
              'name': 'high school',
              'nbinsx': 50,
              'opacity': 1,
              'type': 'histogram',
              'x': array([1407.98666667, 1407.98666667, 1407.98666667, 2208.28767123,
                          2208.28767123, 2208.28767123, 1253.82051282, 1253.82051282,
                          1253.82051282, 1137.12048193, 2056.76744186, 2056.76744186,
                          2056.76744186, 1313.51388889, 1313.51388889, 1313.51388889,
                          1699.82608696, 1699.82608696, 1699.82608696, 1426.62195122,
                          1914.09090909, 1914.09090909, 1914.09090909, 1471.21428571,
                          1471.21428571, 1471.21428571, 1150.6       , 1150.6       ,
                          1150.6       , 1445.64772727, 1445.64772727, 1445.64772727,
                          1306.97333333, 1638.32142857, 1638.32142857, 1638.32142857,
                          1656.19753086, 1656.19753086, 1656.19753086, 1335.34782609,
                          1335.34782609, 1335.34782609, 1846.25      , 1846.25      ,
                          1846.25      , 1503.2       , 1503.2       , 1503.2       ,
                          1639.71428571, 1639.71428571, 1639.71428571, 1214.        ,
                          1214.        , 1214.        , 1387.12359551, 1387.12359551,
                          1387.12359551, 1243.39285714, 1243.39285714, 1243.39285714,
                          1445.81609195, 1445.81609195, 1445.81609195, 1254.85714286,
                          1254.85714286, 1254.85714286, 1099.3253012 , 1106.54878049,
                          1106.54878049, 1106.54878049, 1354.34615385, 1354.34615385,
                          1354.34615385, 1796.02631579, 1796.02631579, 1796.02631579,
                          1092.625     , 1092.625     , 1092.625     , 1410.79104478,
                          1410.79104478, 1410.79104478, 1125.28571429, 1125.28571429,
                          1125.28571429, 1117.52380952, 1117.52380952, 1117.52380952,
                          1541.9382716 , 1541.9382716 , 1541.9382716 ,  877.72941176,
                           877.72941176,  877.72941176, 1520.59090909, 1520.59090909,
                          1520.59090909, 1368.20987654, 1368.20987654, 1368.20987654,
                          1173.46590909, 1173.46590909, 1173.46590909, 1800.92      ,
                          1800.92      , 1800.92      , 1827.16666667, 1827.16666667,
                          1827.16666667, 1649.22666667, 1649.22666667, 1649.22666667,
                          1421.93589744, 1421.93589744, 1421.93589744, 2148.78846154,
                          2148.78846154, 2148.78846154, 1183.39726027, 1183.39726027,
                          1183.39726027,  921.58024691,  921.58024691,  921.58024691,
                          2739.17142857, 2739.17142857, 2739.17142857, 1323.31395349,
                          1323.31395349, 1323.31395349, 1095.98717949, 1095.98717949,
                          1095.98717949, 1292.03947368, 1292.03947368, 1292.03947368,
                          1215.87209302, 1215.87209302, 1215.87209302, 1265.02325581,
                          1265.02325581, 1265.02325581]),
              'xaxis': 'x2',
              'yaxis': 'y2'},
             {'marker': {'color': 'steelblue'},
              'name': 'graduate school',
              'nbinsx': 50,
              'opacity': 1,
              'type': 'histogram',
              'x': array([2531.84      , 2531.84      , 2531.84      , 1968.71428571,
                          1968.71428571, 19

# Anova 

- Var X vs Groups


# Summary Stats
mean + std per grou[]

In [16]:
# =========================================== ANOVA == REACTION TIME ======================================================x
def ANOVA(data, group_var, value_var):
    # Create ANOVA backbone table
    raw_data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']] 
    anova_table = pd.DataFrame(raw_data, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit']) 
    anova_table.set_index('Source of Variation', inplace = True)

    # calculate SSTR and update anova table
    x_bar = data[value_var].mean()
    SSTR = data.groupby(group_var).count() * (data.groupby(group_var).mean() - x_bar)**2
    anova_table['SS']['Between Groups'] = SSTR[value_var].sum()

    # calculate SSE and update anova table
    SSE = (data.groupby(group_var).count() - 1) * data.groupby(group_var).std()**2
    anova_table['SS']['Within Groups'] = SSE[value_var].sum()

    # calculate SSTR and update anova table
    SSTR = SSTR[value_var].sum() + SSE[value_var].sum()
    anova_table['SS']['Total'] = SSTR

    # update degree of freedom
    anova_table['df']['Between Groups'] = data[group_var].nunique() - 1
    anova_table['df']['Within Groups'] = data.shape[0] - data[group_var].nunique()
    anova_table['df']['Total'] = data.shape[0] - 1

    # calculate MS
    anova_table['MS'] = anova_table['SS'] / anova_table['df']

    # calculate F 
    F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
    anova_table['F']['Between Groups'] = F

    # p-value
    anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

    # F critical 
    alpha = 0.05
    # possible types "right-tailed, left-tailed, two-tailed"
    tail_hypothesis_type = "two-tailed"
    if tail_hypothesis_type == "two-tailed":
        alpha /= 2
    anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

    # Final ANOVA Table
    return(anova_table)

In [17]:
ANOVA(data=df, value_var='wcst_RT', group_var='demographics_education_a')

SS   df              MS         F P-value  \
Source of Variation                                                            
Between Groups         9265103.695754    2  4632551.847877  17.51658     0.0   
Within Groups        217127142.647837  821   264466.678012                     
Total                226392246.343591  823   275081.708801                     

                       F crit  
Source of Variation            
Between Groups       3.705504  
Within Groups                  
Total

# Summary Stats

Per group compute the:

- mean
- std
- mean RT

for each task:

- wcst
- navon 
- corsi
- nback
- fitts

In [18]:
df.columns

Index(['nback_status', 'nback_reaction_time_ms', 'fitts_mean_deviation',
       'corsi_block_span', 'navon_level_of_target', 'navon_perc_correct',
       'navon_reaction_time_ms', 'wcst_RT', 'wcst_accuracy',
       'demographics_age_a', 'demographics_gender_a',
       'demographics_handedness_a', 'demographics_education_a',
       'demographics_income_a', 'demographics_computer_hours_a',
       'demographics_age_group', 'demographics_mean_reation_time_ms'],
      dtype='object')

In [19]:
def compute_summary_stats(data=df, value_var='wcst_RT', group_var='demographics_education_a'):
    return data.groupby(group_var).agg({
        'wcst_accuracy':            ['mean', 'std'],
        'wcst_RT':                  'mean',
        'navon_perc_correct':       ['mean', 'std'],
        'navon_reaction_time_ms':   'mean',
        'nback_status':             ['mean', 'std'],
        'nback_reaction_time_ms':   'mean',
        'fitts_mean_deviation':     ['mean', 'std'],
        'corsi_block_span':         ['mean', 'std']    
        })

compute_summary_stats()

wcst_accuracy                wcst_RT  \
                                  mean       std         mean   
demographics_education_a                                        
graduate school               0.666383  0.194748  1792.950860   
high school                   0.767254  0.137067  1460.116058   
university                    0.690600  0.181455  1690.271606   

                         navon_perc_correct           navon_reaction_time_ms  \
                                       mean       std                   mean   
demographics_education_a                                                       
graduate school                    0.586568  0.340555            1382.407996   
high school                        0.763420  0.211709             958.050568   
university                         0.646279  0.304329            1142.875148   

                         nback_status           nback_reaction_time_ms  \
                                 mean       std                   mean   
demographics_education_a                                                 
graduate school              0.719730  0.187126            2194.926892   
high school                  0.801259  0.147912            2324.136960   
university                   0.753257  0.184678            2177.171774   

                         fitts_mean_deviation             corsi_block_span  \
                                         mean         std             mean   
demographics_education_a                                                     
graduate school                   -431.330874  444.327909         2.755435   
high school                       -224.847482  260.012799         5.179856   
university                        -367.324623  376.164444         3.502500   

                                    
                               std  
demographics_education_a            
graduate school           2.761953  
high school               2.058041  
university                2.806186

In [20]:
ed.summary_table.columns

Index(['nback_status', 'nback_reaction_time_ms', 'fitts_mean_deviation',
       'corsi_block_span', 'navon_level_of_target', 'navon_perc_correct',
       'navon_reaction_time_ms', 'wcst_accuracy', 'wcst_RT',
       'demographics_age_a', 'demographics_gender_a',
       'demographics_handedness_a', 'demographics_education_a',
       'demographics_income_a', 'demographics_computer_hours_a',
       'demographics_age_group', 'demographics_mean_reation_time_ms'],
      dtype='object')

In [21]:
def compute_summary_stats(data, value_var='wcst_RT', group_var='demographics_education_a'):
        return data.groupby(group_var).agg({
            'wcst_accuracy':            ['mean', 'std'],
            'wcst_RT':                  'mean',
            'navon_perc_correct':       ['mean', 'std'],
            'navon_reaction_time_ms':   'mean',
            'nback_status':             ['mean', 'std'],
            'nback_reaction_time_ms':   'mean',
            'fitts_mean_deviation':     ['mean', 'std'],
            'corsi_block_span':         ['mean', 'std']    
            })

In [22]:
x = compute_summary_stats(data=ed.summary_table)



In [23]:
from plotly.colors import n_colors

def violin_distribution_plot(
    group_var=('participant',''), groups=participants, value_var=('reaction_time_ms', 'mean'), 
    colors=n_colors('rgb(200, 10, 10)', 'rgb(5, 200, 200)', 10, colortype='rgb'),
    title='Reaction Time Distributions', xaxis={'title':'participants'}, yaxis={'title':'Reaction Times'}, template='plotly_white', 
    legend_title_text='Participant', width=700, height=500):
    
    # ---- extract data ----x
    RTs = []
    for g in groups:
        x = data.loc[data[group_var]==g,][value_var]
        RTs.append(x)

    # ---- sort lists ----x
    srt = np.argsort([np.mean(r) for r in RTs])
    RT = [RTs[s] for s in srt]

    # ---- create figure: violin plots ----x
    traces = []
    for rt in range(len(RT)):
        traces.append(go.Violin(y=RT[rt],
                                name=groups[rt],
                                box_visible=True,
                                meanline_visible=True))

    layout = go.Layout(title=title, xaxis=xaxis, yaxis=yaxis, template=template, legend_title_text=legend_title_text, width=width, height=height)
    fig = go.Figure(data=traces, layout=layout)

    # ---- create figure: distribution plots ----x
    fig_2 = go.Figure()
    for data_line, color, grp in zip(RT, colors, groups):
        fig_2.add_trace(go.Violin(x=data_line, line_color=color, name=grp))


    fig_2.update_traces(orientation='h', side='positive', width=3, points=False)
    fig_2.update_layout(xaxis_showgrid=False, xaxis_zeroline=False, title=title, yaxis=xaxis, xaxis=yaxis, template=template, legend_title_text=legend_title_text, width=width, height=height)


    return({'data':zip(groups, RTs), 'violin_plot':fig, 'distribution_plot':fig_2})


NameError: name 'participants' is not defined

In [24]:
# ---------------- demographic data 
demo_pie_map = {
    'gender_a':     {'dummy_var':'gender_a',        'labels':['male', 'female', 'other'],                       'colors':['steelblue', 'darkred', 'cyan'],                                          'title':'Gender Distribution',     'name':'gender'},
    'education_a':  {'dummy_var':'education_a',     'labels':['university', 'graduate school', 'high school'],  'colors':['rgb(177, 127, 38)', 'rgb(129, 180, 179)', 'rgb(205, 152, 36)'],  'title':'Education Distribution',   'name':'education'},
    'handedness_a': {'dummy_var':'handedness_a',    'labels':['right', 'left', 'ambidextrous'],                 'colors':px.colors.sequential.RdBu,                                         'title':'Handedness Distribution',  'name':'handedness'},
    'age_group':    {'dummy_var':'age_group',       'labels':np.unique(ed.demographics[['age_group']]).tolist(),'colors':px.colors.sequential.GnBu,                                         'title':'Age Distribution',         'name':'age'}
    }
    
demo_continuous_naming = {
     'age_a':                   {'xlab':'Age',                      'ylab':'Count', 'name':'Age Distribution by '},
     'income_a':                {'xlab':'Income',                   'ylab':'Count', 'name':'Income Distribution by '},
     'computer_hours_a':        {'xlab':'Computer hours',           'ylab':'Count', 'name':'Computer Hours Distribution by '},
     'mean_reation_time_ms':    {'xlab':'RT (reaction time (ms))',  'ylab':'Count', 'name':'RT Distribution by '},
}

In [25]:
# ----- all categories descriptors -----x
cats_demographics   = ['gender_a', 'handedness_a', 'education_a', 'age_group']
cats_navon          = [('level_of_target', '')]

# ---- add numerical descriptors ----x
num_demographics = ['age_a','income_a', 'computer_hours_a', 'mean_reation_time_ms']
num_nback = [('block_number', ''), ('score', 'mean'), ('score', 'std'), ('status', 'mean'), ('status', 'std'), ('miss', 'mean'), 
            ('miss', 'std'), ('false_alarm', 'mean'), ('false_alarm', 'std'), ('reaction_time_ms', 'mean'), ('reaction_time_ms', 'std')]
num_navon = [('correct', 'mean'), ('correct',  'std'), ('too_slow', 'mean'), ('too_slow',  'std'), 
            ('reaction_time_ms', 'mean'), ('reaction_time_ms',  'std')]
num_corsi = [('highest_span',  'max'), ('status', 'mean'), ('status',  'std')]
num_fitts = [('delta', 'mean'), ('delta',  'std'), ('status', 'mean')]


categorical_vars = [
    # ------ demographics -----x
    'gender_a', 'education_a', 'handedness_a', 'age_group',
    # ------ Fitts ------x
    'mean_fitts_deviation',
    # ------ Nback ------x
    'nback_status', 'nback_reaction_time_ms',
    
    ]